In [ ]:
# Imports
import json
import math
import tldextract
import numpy as np
import statistics
from collections import defaultdict, Counter, OrderedDict
from urllib.parse import urlparse
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, display
import seaborn as sns
import ast 
import os
%matplotlib inline
from itertools import combinations, product
from IPython.display import display
import ipywidgets as widgets
from IPython.display import display, HTML
from scipy.stats import chi2_contingency, fisher_exact
import statsmodels.formula.api as smf
from statsmodels.stats.proportion import proportions_ztest
from urllib.parse import urlsplit, urlunsplit
from tqdm import tqdm

from urllib.parse import urlparse


In [ ]:
def extract_etld(url):
    if pd.isna(url):
        return None
    return urlparse(url).netloc.split(':')[0]

In [ ]:
db_params = {
    "host": "localhost",
    "user": "cursor",
    "password": "whaTaShame",
    "db": "crawl_analysis_new",
    "cursorclass": pymysql.cursors.DictCursor
}


#### Reading sessions of category global, and computing an IS_VPN column

In [ ]:
def load_rows(conn_params):
    sql = """
    SELECT * FROM crawl_sessions WHERE category NOT LIKE '%validation' and category = 'global' and etld_url_rel <> 'cross-site' and (is_cloudflare IS NULL OR is_cloudflare = 0) 
    """

        
    conn = pymysql.connect(
        host=conn_params['host'],
        user=conn_params['user'],
        password=conn_params['password'],
        db=conn_params['db'],
        cursorclass=pymysql.cursors.DictCursor
    )
    
    try:
        with conn.cursor() as cursor:
            cursor.execute(sql)
            rows = cursor.fetchall()  # list of dicts
            df = pd.DataFrame(rows)
    finally:
        conn.close()
    
    return df

# Load data
print('Loading rows from DB...')
df = load_rows(db_params)
print(f'Loaded {len(df)} rows')

# Normalize some columns
df['etld'] = df['etld'].astype(str).str.lower()
df['category'] = df['category'].astype(str)
df['location'] = df['location'].astype(str)

df['is_VPN'] = df['location'].str.endswith('-VPN')
df['location'] = df['location'].str.replace('-VPN$', '', regex=True)

df

### Keeping only ETLDs-URLs that appear under both IS_VPN = True, and IS_VPN = False, for each country (to allow comparison)

In [ ]:
def normalize_url(url):
    if pd.isna(url):
        return url
    parts = urlsplit(url)
    # drop query and fragment
    return urlunsplit((parts.scheme, parts.netloc, parts.path, '', ''))
df['normalized_url'] = df['url'].apply(normalize_url)


In [ ]:
# List to store results per country
dfs = []

# Loop over countries
for country in df['location'].unique():
    country_df = df[df['location'] == country]
    
    # Find (etld, url) combinations that appear in both VPN and non-VPN
    site_url_counts = country_df.groupby(['etld', 'normalized_url'])['is_VPN'].nunique()
    site_urls_in_both = site_url_counts[site_url_counts == 2].reset_index()[['etld', 'normalized_url']]
    
    # Filter rows to keep only these combinations
    country_filtered = country_df.merge(site_urls_in_both, on=['etld', 'normalized_url'], how='inner')
    
    dfs.append(country_filtered)

# Concatenate all countries
ablation_df = pd.concat(dfs, ignore_index=True)

# Drop duplicates if a row appears multiple times for same (location, etld, url, is_VPN, id)
#ablation_df = ablation_df.drop_duplicates(subset=['location', 'etld', 'url', 'is_VPN', 'id'])

# Now ablation_df contains all rows where (site, url) exists in both VPN and non-VPN
ablation_df

### How many sites/pages we have in the ablation experiment

In [ ]:
country_stats = (
    ablation_df.groupby(['location', 'is_VPN'])
    .agg(
        n_sites=('etld', 'nunique'),
        n_urls=('normalized_url', 'nunique')
    )
    .reset_index()
)

urls_per_site = (
    ablation_df.groupby(['location', 'is_VPN', 'etld'])['normalized_url']
    .nunique()
    .reset_index(name='n_urls_per_site')
)

urls_stats = (
    urls_per_site.groupby(['location', 'is_VPN'])['n_urls_per_site']
    .agg(['mean', 'median', 'std'])
    .reset_index()
)

final_stats = pd.merge(country_stats, urls_stats, on=['location', 'is_VPN'])

pivot_stats = final_stats.pivot(index='location', columns='is_VPN')
pivot_stats.columns = ['_'.join(map(str, col)).strip() for col in pivot_stats.columns]
pivot_stats = pivot_stats.reset_index()


pivot_stats

### Load requests, scripts, js-calls, cookies for each crawling session

In [ ]:
def load_table(table_name, session_ids, conn_params):

    csv_path = os.path.join('utils/ablation', f"{table_name}.csv")
    
    if os.path.exists(csv_path):
        print(f"Loading {table_name} from CSV cache")
        df = pd.read_csv(csv_path)
        return df
    
    if not session_ids:
        return pd.DataFrame()
    
    conn = pymysql.connect(
        host=conn_params['host'],
        user=conn_params['user'],
        password=conn_params['password'],
        db=conn_params['db'],
        cursorclass=pymysql.cursors.DictCursor
    )
    
    all_rows = []
    try:
        with conn.cursor() as cursor:
            for i, sid in enumerate(session_ids, 1):
                sql = f"SELECT * FROM {table_name} WHERE session_id = {sid}"
                cursor.execute(sql)
                rows = cursor.fetchall()
                all_rows.extend(rows)
                
                # Print progress
                if i % 500 == 0 or i == len(session_ids):
                    print(f"{table_name}: Loaded {i}/{len(session_ids)} sessions")
    finally:
        conn.close()

    df = pd.DataFrame(all_rows)

    df.to_csv(csv_path, index=False)
    print(f"Saved {table_name} to {csv_path}")
    
    return df


tables = ['js_calls', 'cookies', 'requests', 'scripts']
dfs = {}


session_ids = ablation_df['id'].tolist()
data_dfs = {}

for table in tables:
    print('Loading:', table)
    
    table_df = load_table(table, session_ids, db_params)
    data_dfs[table] = table_df


## 1. Requests differences

In [ ]:

requests_df = data_dfs['requests'].copy()
requests_df['request_etld'] = requests_df['request_url'].apply(extract_etld)

requests_df['is_complete'] = requests_df['result_status'] == 'complete'

unique_requests = requests_df.drop_duplicates(subset=['session_id', 'request_url'])

request_types = [
    'Script', 'Video', 'Manifest', 'Fetch',
    'XMLHttpRequest', 'Link prefetch', 'Link element', 'Audio'
]

# --- 5. Initialize a dataframe for session metrics ---
ablation_metrics = pd.DataFrame({'id': ablation_df['id']})

# --- 6. Percentage of complete requests ---
complete_counts = requests_df.groupby('session_id')['is_complete'].mean()
ablation_metrics['percentage_of_complete_requests'] = ablation_metrics['id'].map(complete_counts)

# --- 7. n_requests  ---
n_requests = unique_requests.groupby('session_id')['request_url'].count()
ablation_metrics['n_requests'] = ablation_metrics['id'].map(n_requests).fillna(0).astype(int)

# --- 8. n_requests per type  ---
for rtype in request_types:
    col_name = f'n_requests_type_{rtype.replace(" ", "_")}'
    type_counts = unique_requests[unique_requests['request_type'] == rtype] \
        .groupby('session_id')['request_url'].count()
    ablation_metrics[col_name] = ablation_metrics['id'].map(type_counts).fillna(0).astype(int)

# --- 9. Third-party metrics ---
tp_mask = unique_requests['etld_request_url_rel'] == 'cross-site'

# n_requests_to_third_parties
n_tp_requests = unique_requests[tp_mask].groupby('session_id')['request_url'].count()
ablation_metrics['n_requests_to_third_parties'] = ablation_metrics['id'].map(n_tp_requests).fillna(0).astype(int)

# n_distinct_third_party_domains
n_tp_domains = unique_requests[tp_mask].groupby('session_id')['request_etld'].nunique()
ablation_metrics['n_distinct_third_party_domains'] = ablation_metrics['id'].map(n_tp_domains).fillna(0).astype(int)

# --- 10. Tracking metrics ---
tracking_mask = unique_requests['is_tracker'] == True
tp_tracking_mask = tp_mask & tracking_mask

# n_tracking_requests
n_tracking_requests = unique_requests[tracking_mask].groupby('session_id')['request_url'].count()
ablation_metrics['n_tracking_requests'] = ablation_metrics['id'].map(n_tracking_requests).fillna(0).astype(int)

# n_third_party_tracking_requests
n_tp_tracking_requests = unique_requests[tp_tracking_mask].groupby('session_id')['request_url'].count()
ablation_metrics['n_third_party_tracking_requests'] = ablation_metrics['id'].map(n_tp_tracking_requests).fillna(0).astype(int)

# n_distinct_third_party_tracking_domains
n_tp_tracking_domains = unique_requests[tp_tracking_mask].groupby('session_id')['request_etld'].nunique()
ablation_metrics['n_distinct_third_party_tracking_domains'] = ablation_metrics['id'].map(n_tp_tracking_domains).fillna(0).astype(int)

# Merge back to ablation_df ---
ablation_df = ablation_df.drop(columns=[c for c in ablation_metrics.columns if c != 'id'], errors='ignore')
ablation_df = ablation_df.merge(ablation_metrics, on='id', how='left')


In [ ]:
pd.set_option('display.max_columns', None)

numeric_cols = [
    'n_requests',
    'n_requests_to_third_parties',
    'n_distinct_third_party_domains',
    'n_tracking_requests',
    'n_third_party_tracking_requests',
    'n_distinct_third_party_tracking_domains',
    'percentage_of_complete_requests'
] #+ [col for col in ablation_df.columns if col.startswith('n_requests_type_')]

# --- 2. Aggregate per location and VPN status ---
agg_location_vpn = ablation_df.groupby(['location', 'is_VPN'])[numeric_cols] \
    .agg(['mean', 'median']) \
    .reset_index()

# --- 3. Flatten MultiIndex columns ---
agg_location_vpn.columns = [
    '_'.join(filter(None, col)).strip() if isinstance(col, tuple) else col
    for col in agg_location_vpn.columns
]

agg_location_vpn

In [ ]:
# Choose the group level (e.g., by location, VPN, or both)
group_cols = ['location', 'is_VPN']  # adjust as needed

# Copy relevant columns
df = ablation_df.copy()

# Binary flag: site has at least one tracker
df['has_tracker'] = df['n_tracking_requests'] > 0

# Aggregation per group
tracker_stats = df.groupby(group_cols).agg(
    # Percentage of sites with ≥1 tracker
    pct_sites_with_tracker = ('has_tracker', lambda x: 100 * x.mean()),

    # Median number of third-party trackers per site
    median_third_party_trackers = ('n_third_party_tracking_requests', 'median'),

    # Average number of third-party trackers per site
    avg_third_party_trackers = ('n_third_party_tracking_requests', 'mean'),

    # Mean number of requests per site
    n_requests_mean = ('n_requests', 'mean')
).reset_index()

tracker_stats


# 2. Scripts differences

In [ ]:
scripts_df = data_dfs['scripts'].copy()

script_metrics = pd.DataFrame({'id': ablation_df['id']})

# --- 4. Total number of scripts ---
n_scripts = scripts_df.groupby('session_id')['script_src'].count()
script_metrics['number_scripts'] = script_metrics['id'].map(n_scripts).fillna(0).astype(int)

tp_mask = scripts_df['etld_script_src_rel'] == 'cross-site'
n_tp_scripts = scripts_df[tp_mask].groupby('session_id')['script_src'].count()
script_metrics['number_third_party_scripts'] = script_metrics['id'].map(n_tp_scripts).fillna(0).astype(int)

# --- 6. Number of main-frame scripts ---
main_frame_mask = scripts_df['frame_main'] == 1
n_main_frame_scripts = scripts_df[main_frame_mask].groupby('session_id')['script_src'].count()
script_metrics['number_main_frame_scripts'] = script_metrics['id'].map(n_main_frame_scripts).fillna(0).astype(int)

# --- 7. Number of non-main-frame scripts ---
non_main_frame_mask = scripts_df['frame_main'] == 0
n_non_main_frame_scripts = scripts_df[non_main_frame_mask].groupby('session_id')['script_src'].count()
script_metrics['number_non_main_frame_scripts'] = script_metrics['id'].map(n_non_main_frame_scripts).fillna(0).astype(int)

# --- 8. Merge back to ablation_df ---
ablation_df = ablation_df.merge(script_metrics, on='id', how='left')

In [ ]:
ablation_df

In [ ]:
numeric_cols = [
    'number_scripts',
    'number_third_party_scripts',
    'number_main_frame_scripts',
    'number_non_main_frame_scripts'
]

# Aggregate per location and VPN status
agg_location_vpn = ablation_df.groupby(['location', 'is_VPN'])[numeric_cols] \
    .agg(['mean']) \
    .reset_index()

# Flatten MultiIndex columns
agg_location_vpn.columns = [
    '_'.join(filter(None, col)).strip() if isinstance(col, tuple) else col
    for col in agg_location_vpn.columns
]

agg_location_vpn

In [ ]:
# Columns to summarize
numeric_cols = [
    'number_third_party_scripts',
    'number_non_main_frame_scripts'  # this is the column in ablation_df
]

# Aggregate per location and VPN status
agg_scripts_summary = ablation_df.groupby(['location', 'is_VPN'])[numeric_cols] \
    .agg(['mean']) \
    .reset_index()

# Flatten MultiIndex columns
agg_scripts_summary.columns = [
    '_'.join(filter(None, col)).strip() if isinstance(col, tuple) else col
    for col in agg_scripts_summary.columns
]

agg_scripts_summary


## 4. JS Calls

In [ ]:
# Copy JS calls DataFrame
js_df = data_dfs['js_calls'].copy()  # adjust key if your dictionary uses a different name

# Initialize per-session metrics dataframe
js_metrics = pd.DataFrame({'id': ablation_df['id']})

# --- 1. Total number of JS calls ---
n_js_calls = js_df.groupby('session_id')['caller_url'].count()
js_metrics['number_js_calls'] = js_metrics['id'].map(n_js_calls).fillna(0).astype(int)

# --- 2. Number of third-party JS calls ---
tp_mask = js_df['etld_caller_url_rel'] == 'cross-site'
n_tp_js_calls = js_df[tp_mask].groupby('session_id')['caller_url'].count()
js_metrics['number_third_party_js_calls'] = js_metrics['id'].map(n_tp_js_calls).fillna(0).astype(int)

# --- 3. Number of non-main-frame JS calls ---
non_main_frame_mask = js_df['context_main'] == 0
n_non_main_js_calls = js_df[non_main_frame_mask].groupby('session_id')['caller_url'].count()
js_metrics['number_non_main_frame_js_calls'] = js_metrics['id'].map(n_non_main_js_calls).fillna(0).astype(int)

# --- 4. Merge back to ablation_df ---
ablation_df = ablation_df.merge(js_metrics, on='id', how='left')

In [ ]:
numeric_cols = [
    'number_js_calls',
    'number_third_party_js_calls',
    'number_non_main_frame_js_calls',
]

# Aggregate per location and VPN status
agg_location_vpn = ablation_df.groupby(['location', 'is_VPN'])[numeric_cols] \
    .agg(['mean']) \
    .reset_index()

# Flatten MultiIndex columns
agg_location_vpn.columns = [
    '_'.join(filter(None, col)).strip() if isinstance(col, tuple) else col
    for col in agg_location_vpn.columns
]

agg_location_vpn

## Fingerprinting

In [ ]:
group_cols = ['location', 'is_VPN']  # adjust as needed

# Function to compute percentage of unique (etld, url) that are fingerprinting
def fingerprinting_percentage(group):
    total_sites = group[['etld', 'url']].drop_duplicates().shape[0]
    fp_sites = group[group['is_fingerprinting'] == True][['etld', 'url']].drop_duplicates().shape[0]
    return 100 * fp_sites / total_sites if total_sites > 0 else 0

# Apply per group
fp_stats = ablation_df.groupby(group_cols).apply(fingerprinting_percentage).reset_index(name='pct_fingerprinting_sites')

fp_stats



# In the paper

In [ ]:
# --- Columns to select from tracker_stats ---
tracker_cols = [
    'location',
    'is_VPN',
    'pct_sites_with_tracker',
    'avg_third_party_trackers',
    'n_requests_mean'
]

# --- Columns to select from scripts summary (after flattening) ---
scripts_cols = [
    'location',
    'is_VPN',
    'number_third_party_scripts_mean',
    'number_non_main_frame_scripts_mean'
]

# --- Merge tracker and scripts summaries ---
PAPER_RESULT_DF = tracker_stats[tracker_cols].merge(
    agg_scripts_summary[scripts_cols],
    on=['location', 'is_VPN'],
    how='left'
)

# --- Optional: rename script columns to remove '_mean' suffix for cleaner presentation ---
PAPER_RESULT_DF = PAPER_RESULT_DF.rename(columns={
    'number_third_party_scripts_mean': 'number_third_party_scripts',
    'number_non_main_frame_scripts_mean': 'number_non_main_frame_scripts'
})

PAPER_RESULT_DF
